In [2]:
import logging
import os
from pathlib import Path

import pandas as pd
from tqdm import tqdm

if Path.cwd().name == "notebooks":
    os.chdir("..")

from src.evaluate import get_predictions_with_gt, save_analysis_results, store_run_metrics_df
from src.formatting import format_storing_pathes_from_run_path

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
prompts = [
    "direct_entity",
    "direct_entity_ontological",
    "sequential_hierarchy",
    "sequential_hierarchy_ontological",
    "direct_entity_with_synonyms",
    "sequential_hierarchy_with_synonyms",
]

runs = [
    ("2025-05-11_00-15-12", "gemini-1.5-flash", "omim-ordo", "bioml-2024"),
    ("2025-05-11_00-12-18", "gemini-1.5-flash", "ncit-doid", "bioml-2024"),
    ("2025-05-11_00-06-56", "gemini-1.5-flash", "snomed-ncit.pharm", "bioml-2024"),
    ("2025-05-11_00-02-13", "gemini-1.5-flash", "snomed-ncit.neoplas", "bioml-2024"),
    ("2025-05-10_23-57-23", "gemini-1.5-flash", "snomed-fma.body", "bioml-2024"),
    ("2025-05-10_23-56-32", "gemini-1.5-flash", "human-mouse", "anatomy"),
    ("2025-05-10_23-41-12", "gemini-1.5-flash", "fma-snomed", "largebio"),
    ("2025-05-10_23-38-28", "gemini-1.5-flash", "fma-nci", "largebio"),
    ("2025-05-10_23-26-32", "gemini-1.5-flash", "snomed-nci", "largebio"),
    ("2025-05-10_21-38-48", "gemini-2.5-flash-preview-04-17", "fma-snomed", "largebio"),
    ("2025-05-10_21-26-58", "gemini-2.5-flash-preview-04-17", "fma-nci", "largebio"),
    ("2025-05-10_20-48-45", "gemini-2.5-flash-preview-04-17", "snomed-nci", "largebio"),
    ("2025-05-10_20-31-31", "gemini-2.0-flash", "omim-ordo", "bioml-2024"),
    ("2025-05-10_20-26-50", "gemini-2.0-flash", "ncit-doid", "bioml-2024"),
    ("2025-05-10_20-24-42", "gemini-2.5-flash-preview-04-17", "omim-ordo", "bioml-2024"),
    ("2025-05-10_20-18-58", "gemini-2.0-flash", "snomed-ncit.pharm", "bioml-2024"),
    ("2025-05-10_20-12-48", "gemini-2.5-flash-preview-04-17", "ncit-doid", "bioml-2024"),
    ("2025-05-10_20-12-05", "gemini-2.0-flash", "snomed-ncit.neoplas", "bioml-2024"),
    ("2025-05-10_20-04-10", "gemini-2.0-flash", "snomed-fma.body", "bioml-2024"),
    ("2025-05-10_20-02-37", "gemini-2.0-flash", "human-mouse", "anatomy"),
    ("2025-05-10_19-59-37", "gemini-2.5-flash-preview-04-17", "snomed-ncit.pharm", "bioml-2024"),
    ("2025-05-10_19-58-03", "gemini-2.0-flash-lite", "omim-ordo", "bioml-2024"),
    ("2025-05-10_19-54-15", "gemini-2.0-flash-lite", "ncit-doid", "bioml-2024"),
    ("2025-05-10_19-47-56", "gemini-2.0-flash-lite", "snomed-ncit.pharm", "bioml-2024"),
    ("2025-05-10_19-47-03", "gemini-2.5-flash-preview-04-17", "snomed-ncit.neoplas", "bioml-2024"),
    ("2025-05-10_19-42-52", "gemini-2.0-flash-lite", "snomed-ncit.neoplas", "bioml-2024"),
    ("2025-05-10_19-37-02", "gemini-2.0-flash-lite", "snomed-fma.body", "bioml-2024"),
    ("2025-05-10_19-35-50", "gemini-2.0-flash-lite", "human-mouse", "anatomy"),
    ("2025-05-10_19-32-25", "gemini-2.5-flash-preview-04-17", "snomed-fma.body", "bioml-2024"),
    ("2025-05-10_19-29-40", "gemini-2.5-flash-preview-04-17", "human-mouse", "anatomy"),
    ("2025-05-10_16-09-58", "gemini-2.0-flash", "fma-snomed", "largebio"),
    ("2025-05-10_16-07-22", "gemini-2.0-flash-lite", "fma-snomed", "largebio"),
    ("2025-05-10_16-05-15", "gemini-2.0-flash", "fma-nci", "largebio"),
    ("2025-05-10_16-03-43", "gemini-2.0-flash-lite", "fma-nci", "largebio"),
    ("2025-05-10_15-46-10", "gemini-2.0-flash-lite", "snomed-nci", "largebio"),
    ("2025-05-10_15-44-53", "gemini-2.0-flash", "snomed-nci", "largebio"),
    ("2025-03-28_10-15-04", "gpt-4o-mini", "fma-nci", "largebio"),
    ("2025-03-19_23-32-20", "gpt-4o-mini", "omim-ordo", "bioml-2024"),
    ("2025-03-19_22-17-52", "gpt-4o-mini", "ncit-doid", "bioml-2024"),
    ("2025-03-19_19-48-41", "gpt-4o-mini", "human-mouse", "anatomy"),
    ("2025-03-14_00-40-55", "gpt-4o-mini", "ncit-doid", "bioml-2024"),
    ("2025-03-12_17-21-53", "gpt-4o-mini", "human-mouse", "anatomy"),
    ("2025-03-12_14-43-32", "gpt-4o-mini", "omim-ordo", "bioml-2024"),
]

In [ ]:
metrics = pd.read_csv("./runs/all_runs_metrics.csv")

# for row in metrics.iterrows():
for run, model, subset, dataset in tqdm(runs):
    for exp in prompts:
        try:
            run_path = Path("runs") / run
            subset_path = (
                Path(f"data/{dataset}/{subset}")
                / f"{dataset}-{subset}-oasystem_mappings_to_ask_oracle_user_llm_reduced.txt"
            )
            prediction_path, stats_path, diagram_path = format_storing_pathes_from_run_path(
                run_path, subset, model, exp
            )
            preds = pd.read_csv(prediction_path)
            reduced = pd.read_csv(subset_path, sep="|")

            pairs = set(map(tuple, reduced.iloc[:, [0, 1]].values))
            pairs.update({(t, s) for s, t in pairs})
            filtered_preds = preds[preds.apply(lambda x: (x["Source"], x["Target"]) in pairs, axis=1)]  # noqa: B023

            filtered_preds.to_csv(str(prediction_path).split(".csv")[0] + "_reduced.csv", index=False)

        except Exception as e:  # noqa: BLE001
            logging.info(f"Error processing {run_path} | {subset} | {model} | {exp}: {e}")
            continue

        prediction_path, stats_path, diagram_path = format_storing_pathes_from_run_path(
            run_path, subset, model, exp, suffix="_reduced"
        )
        predictions = get_predictions_with_gt(run_path, dataset, subset, model, exp, suffix="_reduced")
        save_analysis_results(
            predictions,
            print_results=False,
            plot_confusion_matrix=False,
            subtitle=f"{subset}: {model} {exp} | ",
            cm_save_path=diagram_path,
            stats_path=stats_path,
        )

    store_run_metrics_df(prompts, run_path, dataset, subset, model, suffix="_reduced")

  0%|          | 0/43 [00:00<?, ?it/s]

 93%|█████████▎| 40/43 [00:06<00:00,  7.97it/s]

File not found for direct_entity in runs/2025-03-19_23-32-20
File not found for direct_entity_ontological in runs/2025-03-19_23-32-20
File not found for sequential_hierarchy in runs/2025-03-19_23-32-20
File not found for sequential_hierarchy_ontological in runs/2025-03-19_23-32-20
File not found for direct_entity in runs/2025-03-19_22-17-52
File not found for direct_entity_ontological in runs/2025-03-19_22-17-52
File not found for sequential_hierarchy in runs/2025-03-19_22-17-52
File not found for sequential_hierarchy_ontological in runs/2025-03-19_22-17-52
File not found for direct_entity in runs/2025-03-19_19-48-41
File not found for direct_entity_ontological in runs/2025-03-19_19-48-41
File not found for sequential_hierarchy in runs/2025-03-19_19-48-41
File not found for sequential_hierarchy_ontological in runs/2025-03-19_19-48-41
File not found for direct_entity_with_synonyms in runs/2025-03-14_00-40-55
File not found for sequential_hierarchy_with_synonyms in runs/2025-03-14_00-40-

100%|██████████| 43/43 [00:07<00:00,  5.96it/s]

File not found for direct_entity_with_synonyms in runs/2025-03-12_17-21-53
File not found for sequential_hierarchy_with_synonyms in runs/2025-03-12_17-21-53
File not found for direct_entity_with_synonyms in runs/2025-03-12_14-43-32
File not found for sequential_hierarchy_with_synonyms in runs/2025-03-12_14-43-32
